In [1]:
import csv
import os
import cv2
import numpy as np
import random
from shutil import copyfile

In [2]:
dataset_root = "/media/arg_ws3/5E703E3A703E18EB/data/sim2real"
mm_root = "/media/arg_ws3/5E703E3A703E18EB/data/mm_unity_new/"
real_root = os.path.join(mm_root, "real_totes")
unity_totes_root = os.path.join(mm_root, "unity_totes")
unity_objects_root = os.path.join(mm_root, "objects")

dataset_real_tote_root = os.path.join(dataset_root, "real_tote")
dataset_unity_tote_root = os.path.join(dataset_root, "unity_tote")
dataset_image_root = os.path.join(dataset_root, "image")
dataset_mask_root = os.path.join(dataset_root, "mask")

if not os.path.exists(dataset_root):
    os.makedirs(dataset_root)
if not os.path.exists(dataset_real_tote_root):
    os.makedirs(dataset_real_tote_root)
if not os.path.exists(dataset_unity_tote_root):
    os.makedirs(dataset_unity_tote_root)
if not os.path.exists(dataset_image_root):
    os.makedirs(dataset_image_root)
if not os.path.exists(dataset_mask_root):
    os.makedirs(dataset_mask_root)

In [3]:
cls_list = os.listdir(unity_objects_root)

In [4]:
def get_box_path(dirpath, filename):
    box_folder = ''
    if '108' in dirpath.split('/')[-2]:
        box_folder = 'box_108'
    elif '72' in dirpath.split('/')[-2]:
        box_folder = 'box_72'
    elif '116' in dirpath.split('/')[-2]:
        box_folder = 'box_116'
    box_path = os.path.join(unity_totes_root, box_folder, dirpath.split('/')[-1], filename)
    return box_path

In [5]:
data_list = []
for (dirpath, dirnames, filenames) in os.walk(unity_objects_root):
    for filename in filenames:
        if filename.endswith('.jpg') or filename.endswith('_original.png'): 
            img_path = os.path.join(dirpath[len(unity_objects_root):], filename)
            mask_path = img_path.replace('_original.png', '_seg.png')
            cls = img_path.split('/')[1]
            cls_idx = cls_list.index(cls) + 1
            img = cv2.imread(unity_objects_root + img_path)
            mask_img = cv2.imread(unity_objects_root + mask_path)[:,:,2]
            mask_img[mask_img > 0] = cls_idx
            img_name = img_path.replace('/', '_')[1:]
            mask_path = mask_path.replace('/', '_')[1:]
            box_path = get_box_path(dirpath, filename)
            box_name = box_path[len(unity_totes_root):].replace('/', '_')[1:]
            if not os.path.isfile(dataset_unity_tote_root + "/" + box_name):
                copyfile(box_path, dataset_unity_tote_root + "/" + box_name)
            copyfile(unity_objects_root + img_path, dataset_image_root + "/" + img_name)
            cv2.imwrite(dataset_mask_root + "/" + mask_path, mask_img)
            l = ["image/" + img_name, "mask/" + mask_path, "unity_tote/" + box_name]
            data_list.append(l)
            #print(dataset_image_root + "/" + img_name)
            #print(dataset_mask_root + "/" + mask_path)
            #print(mask_img[mask_img!=0])
            #break

In [6]:
random.shuffle(data_list)

with open(os.path.join(dataset_root, 'menu.csv'), 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for item in data_list:
        writer.writerow([item[0], item[1], item[2]])

# Create Tote Sim2Real CSV

In [14]:
def createCSV(path, name, ratio = 0.9):
    data_list = []
    training_list = []
    test_list = []
    count = 0
        
    for (dirpath, dirnames, filenames) in os.walk(path):
        for filename in filenames:
            data_list.append(os.path.join(dirpath[len(dataset_root):], filename))
    random.shuffle(data_list)
    
    
    for data in data_list:
        if count < len(data_list)*ratio:
            training_list.append(data)
        else:
            test_list.append(data)
        count = count + 1
        
    with open(os.path.join(dataset_root, name + '_train.csv'), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for item in training_list:
            writer.writerow([item])
    with open(os.path.join(dataset_root, name + '_test.csv'), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for item in test_list:
            writer.writerow([item])

In [15]:
createCSV(dataset_real_tote_root, "real_totes")
createCSV(dataset_unity_tote_root, "unity_totes")